In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

%load_ext autoreload
%autoreload 2

In [ ]:
from crawler import async_cache_pages, urls_list
from features import construct_structural_features, feature_functions

In [ ]:
# edX does not work as some content seems to be dynamically downloaded
folders = ["general2"]

positives, negatives = urls_list(folders)
async_cache_pages(positives + negatives)

In [ ]:
urls = [positives[0], negatives[0]]
labels = [True, False]
features = ["button_count", "a_count"]

construct_structural_features(urls, labels, features)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
all_features = sorted(list(feature_functions.keys()))

In [ ]:
def cross_val(df, selected_features, estimator=RandomForestClassifier(), cv=3):
    X = df[selected_features]
    y = df["label"]
    
    return cross_val_score(estimator, X, y, cv=cv, scoring='f1')

In [ ]:
urls_pos, urls_neg = urls_list(["general2"])
urls = urls_pos + urls_neg
labels = [True] * len(urls_pos) + [False] * len(urls_neg)

results_means = []
results_stds = []

df = construct_structural_features(urls, labels, all_features)

# iterate over all features and train on this feature alone
for feature in all_features:
    # Cross val with the selected feature alone
    scores = cross_val(df, [feature])
    
    results_means.append(np.mean(scores))
    results_stds.append(np.std(scores))

In [ ]:
# Same thing, but with same ratio of positive and negative examples

import random
random.seed(0)

urls_pos, urls_neg = urls_list(["general2"])

# Same number of negative and positive examples
urls = urls_pos + random.sample(urls_neg, len(urls_pos))
labels = [True] * len(urls_pos) + [False] * len(urls_pos)

results_unbiased_means = []
results_unbiased_stds = []

df_unbiased = construct_structural_features(urls, labels, all_features)

# iterate over all features and train on one feature alone
for feature in all_features:
    # Cross val with the selected feature alone
    scores = cross_val(df_unbiased, [feature])
    
    results_unbiased_means.append(np.mean(scores))
    results_unbiased_stds.append(np.std(scores))

In [ ]:
import os

index = np.arange(len(results_means))
fig, ax = plt.subplots(figsize=(18, 8))
bar_width = 0.35
axis_font_size = 18

plt.bar(index, results_means, bar_width, yerr=results_stds, color='tab:red', label="Biased estimator")
plt.bar(index + bar_width, results_unbiased_means, bar_width, yerr=results_unbiased_stds, label="Unbiased estimator")

plt.xlabel("Feature", fontsize=axis_font_size)
plt.ylabel("Cross validation f1 score", fontsize=axis_font_size)
plt.xticks(index + bar_width / 2, all_features, fontsize=14)

#plt.axhline(y=0.5, color='tab:brown', alpha=0.7, linewidth=1)
plt.legend(fontsize=15)

plt.savefig(os.path.join("figures", "feature_score_biased_unbiased_f1"))

In [ ]:
rf = RandomForestClassifier()

X = df_unbiased.drop("label", axis=1)
y = df_unbiased["label"]

rf.fit(X, y)

In [ ]:
feature_importances = rf.feature_importances_
feature_importances_stds = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

index = np.arange(len(results_means))
fig, ax = plt.subplots(figsize=(18, 8))
bar_width = 0.35
axis_font_size = 18

plt.bar(index, results_unbiased_means, bar_width, yerr=results_unbiased_stds, label="Unbiased estimator")
plt.bar(index + bar_width, feature_importances, bar_width, color='tab:gray', yerr=feature_importances_stds, label="Feature importances")

plt.xlabel("Feature", fontsize=axis_font_size)
plt.ylabel("Cross validation f1 score / Feature importance", fontsize=axis_font_size)
plt.xticks(index + bar_width / 2, all_features, fontsize=14)

#plt.axhline(y=0.5, color='tab:brown', alpha=0.7, linewidth=1)
plt.legend(fontsize=15)

plt.savefig(os.path.join("figures", "feature_importance_and_feature_score"))

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def training_score(df, selected_features, estimator=RandomForestClassifier()):
    X = df[selected_features]
    y = df["label"]
    
    estimator.fit(X, y)
    y_pred = estimator.predict(X)
    
    return f1_score(y, y_pred)

In [ ]:
good_features = ["a_count", "button_count", "h1_count", "h2_count", "h3_count"]

In [ ]:
n_estims_list = range(1, 21)
score_means = []
score_stds = []
train_scores = []
features = good_features

for n_estims in n_estims_list:
    scores = cross_val(df_unbiased, features, estimator=RandomForestClassifier(n_estimators=n_estims))
    
    score_means.append(np.mean(scores))
    score_stds.append(np.std(scores))
    
    train_score = training_score(df_unbiased, features, estimator=RandomForestClassifier(n_estimators=n_estims))
    train_scores.append(train_score)

In [ ]:
plt.plot(n_estims_list, score_means)
plt.plot(n_estims_list, train_scores)

SVM is not supposed to work well, because ". SVMs do not perform well on highly skewed/imbalanced data sets. These are training data sets in which the number of samples that fall in one of the classes far outnumber those that are a member of the other class. Customer churn data sets are typically in this group because when you collect the training set, among a million customers during a particular time period, there would be very few who have actually churned. SMOTING is used to generate artificial samples in the minority class to balance the data set. On the other hand, Logistic Regression is good at handling skewed data sets."

In [ ]:
from sklearn import svm

kernels = ['linear', 'sigmoid', 'rbf']
score_means = []
score_stds = []
train_scores = []
features = good_features

for kernel in kernels:
    scores = cross_val(df_unbiased, features, estimator=svm.SVC(kernel=kernel))
    
    score_means.append(np.mean(scores))
    score_stds.append(np.std(scores))
    
    train_score = training_score(df_unbiased, features, estimator=svm.SVC(kernel=kernel))
    train_scores.append(train_score)
    
plt.plot(score_means)
plt.plot(train_scores)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

n_neigbours = range(1, 40)
score_means = []
score_stds = []
train_scores = []
features = good_features

for n in n_neigbours:
    scores = cross_val(df_unbiased, features, estimator=KNeighborsClassifier(n_neighbors=n))
    
    score_means.append(np.mean(scores))
    score_stds.append(np.std(scores))
    
    train_score = training_score(df_unbiased, features, estimator=KNeighborsClassifier(n_neighbors=n))
    train_scores.append(train_score)
    
plt.plot(n_neigbours, score_means)
plt.plot(n_neigbours, train_scores)

In [ ]:
from sklearn.neural_network import MLPClassifier

layers = range(1, 15)
score_means = []
score_stds = []
train_scores = []
features = good_features

for layer in layers:
    scores = cross_val(df_unbiased, features, estimator=MLPClassifier(hidden_layer_sizes=(30,)*layer, random_state=1))
    
    score_means.append(np.mean(scores))
    score_stds.append(np.std(scores))
    
    train_score = training_score(df_unbiased, features, estimator=MLPClassifier(hidden_layer_sizes=(30,)*layer, random_state=1))
    train_scores.append(train_score)
    
plt.plot(layers, score_means)
plt.plot(layers, train_scores)

In [ ]:
from sklearn.linear_model import LogisticRegression

reg_terms = np.arange(0.1, 1, 0.05)
score_means = []
score_stds = []
train_scores = []
features = good_features

for reg in reg_terms:
    scores = cross_val(df_unbiased, features, estimator=LogisticRegression(C=reg, random_state=1))
    
    score_means.append(np.mean(scores))
    score_stds.append(np.std(scores))
    
    train_score = training_score(df_unbiased, features, estimator=LogisticRegression(C=reg, random_state=1))
    train_scores.append(train_score)
    
plt.plot(reg_terms, score_means)
plt.plot(reg_terms, train_scores)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

alphas = np.arange(0.1, 1000, 5)
score_means = []
score_stds = []
train_scores = []
features = good_features

for alpha in alphas:
    scores = cross_val(df_unbiased, features, estimator=MultinomialNB(alpha=alpha))
    
    score_means.append(np.mean(scores))
    score_stds.append(np.std(scores))
    
    train_score = training_score(df_unbiased, features, estimator=MultinomialNB(alpha=alpha))
    train_scores.append(train_score)
    
plt.plot(alphas, score_means)
plt.plot(alphas, train_scores)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
def analyse_classification(X, y, predictor, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
    predictor.fit(X_train, y_train)
    y_pred = predictor.predict(X_test)
    
    print(metrics.classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
X = df[good_features]
y = df["label"]

print(cross_val_score(KNeighborsClassifier(n_neighbors=5), X, y))
print(cross_val_score(KNeighborsClassifier(n_neighbors=5), X, y, scoring='f1'))

In [ ]:
analyse_classification(X, y, KNeighborsClassifier(n_neighbors=5))

In [ ]:
X = df_unbiased[good_features]
y = df_unbiased["label"]

print(cross_val_score(KNeighborsClassifier(n_neighbors=5), X, y))
print(cross_val_score(KNeighborsClassifier(n_neighbors=5), X, y, scoring='f1'))

In [ ]:
analyse_classification(X, y, KNeighborsClassifier(n_neighbors=5))